In [ ]:
import os

def list_files_and_dirs(start_path):
    for root, dirs, files in os.walk(start_path):
        level = root.replace(start_path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        sub_indent = ' ' * 4 * (level + 1)
        for f in files:
            if str(f).endswith('.srt'):
                print(f"{sub_indent}{f}")   

# Replace 'your_directory_path' with the path to the directory you want to list
list_files_and_dirs('/home/al/Documents/courses/Coursera - Financial Engineering and Risk Management Part I')

In [ ]:
below is the output. now i want to make that .srt to ebook in markdown format. 

```
Coursera - Financial Engineering and Risk Management Part I/
    001.Course Overview/
        001. Course Overview.srt
    002.Basics of Fixed Income Securities/
        002. Introduction to No-arbitrage.srt
        003. Interest Rates and Fixed Income Instruments.srt
    003.Basic Fixed Income Instruments/
        004. Floating Rate Bonds and Term Structure of Interest Rates.srt
        005. Forward Contracts.srt
```

it should be become 

```

# 001.Course Overview 

## 001. Course Overview

<content of 001. Course Overview>

# 002.Basics of Fixed Income Securities

and so on 

```

In [ ]:
import os
import pysrt

def srt_to_markdown(start_path, output_file):
    with open(output_file, 'w', encoding='utf-8') as md_file:
        for root, dirs, files in os.walk(start_path):
            level = root.replace(start_path, '').count(os.sep)
            indent = '#' * (level + 1)
            
            if root != start_path:
                md_file.write(f"{indent} {os.path.basename(root)}\n\n")
            
            for file in sorted(files):
                if file.endswith('.srt'):
                    srt_path = os.path.join(root, file)
                    srt_content = pysrt.open(srt_path)
                    
                    file_name = os.path.splitext(file)[0]
                    md_file.write(f"{indent}# {file_name}\n\n")

                    out_text = ""
                    
                    for i, subtitle in enumerate(srt_content):
                        sbt_text = subtitle.text.replace('\n', ' ')
                        if i == len(srt_content) - 1:
                            # md_file.write(f"{subtitle.text}\n\n")
                            out_text += f"{sbt_text}\n\n"
                        else:
                            # md_file.write(f"{subtitle.text} ")
                            out_text += f"{sbt_text}"
                    
                    md_file.write(out_text)


if __name__ == "__main__":
    start_path = '/home/al/Documents/courses/Coursera - Financial Engineering and Risk Management Part I'
    output_file = 'Financial_Engineering_and_Risk_Management_Part_I.md'
    srt_to_markdown(start_path, output_file)

Fix metadata song

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
CLIENT_ID = os.getenv('CLIENT_ID')
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

In [ ]:
import re

# Function to clean file names
def clean_file_name(file_name):
    # Remove invalid characters from the file name
    cleaned_name = re.sub(r'[<>:"/\\|?*]', '', file_name)
    return cleaned_name

In [ ]:
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from mutagen.id3 import ID3, TIT2, TPE1, TALB

# Set up the Spotipy client
client_credentials_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Function to update metadata
def update_metadata(file_path):
    # Extract song name from the file name
    song_name = os.path.splitext(os.path.basename(file_path))[0]
    
    # Search for the song using the Spotify API
    results = sp.search(q=song_name, limit=1, type='track')
    
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        
        # Extract the relevant metadata
        title = track['name']
        artist = track['artists'][0]['name']
        album = track['album']['name']
        
        # Update the metadata of the MP3 file
        try:
            audio = ID3(file_path)
            audio['TIT2'] = TIT2(encoding=3, text=title)
            audio['TPE1'] = TPE1(encoding=3, text=artist)
            audio['TALB'] = TALB(encoding=3, text=album)
            audio.save()
        except:
            print(f"No results found for '{song_name}'.")
        
        # Generate the new file name
        new_file_name = f"{artist} - {title}.mp3"
        new_file_name = clean_file_name(new_file_name)  # Clean the new file name
        new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
        
        # Check if the new file path already exists
        counter = 1
        while os.path.exists(new_file_path):
            file_name, file_extension = os.path.splitext(new_file_name)
            new_file_name = f"{file_name} ({counter}){file_extension}"
            new_file_path = os.path.join(os.path.dirname(file_path), new_file_name)
            counter += 1

        # Rename the file
        os.rename(file_path, new_file_path)

        # Store the original and new file names
        with open('file_names_mapping.txt', 'a') as file:
            file.write(f"{file_name},{new_file_name}\n")

    else:
        print(f"No results found for '{song_name}'.")

# Specify the root directory where your MP3 files are located
root_directory = 'D:\Music'

# Recursively traverse the directory and update metadata and file names
for dirpath, dirnames, filenames in os.walk(root_directory):
    for filename in filenames:
        if filename.endswith('.mp3'):
            file_path = os.path.join(dirpath, filename)
            update_metadata(file_path)


In [ ]:
# Undo the changes and revert the file names
def undo_changes():
    # Read the file_names_mapping.txt to get the original and new file names
    with open('file_names_mapping.txt', 'r') as file:
        mappings = file.readlines()

    # Iterate through the mappings and revert the file names
    for mapping in mappings:
        mapping = mapping.strip().split(',')
        original_file_name, new_file_name = mapping[0], mapping[1]
        original_file_path = os.path.join(root_directory, original_file_name)
        new_file_path = os.path.join(root_directory, new_file_name)

        # Rename the file back to the original file name
        os.rename(new_file_path, original_file_path)

    # Remove the file_names_mapping.txt file
    os.remove('file_names_mapping.txt')

In [ ]:
import os

def print_directory_structure_with_vtt(directory):
    """
    Traverse the given directory, print the directory structure with headings,
    and print only the file names ending with ".en.vtt".
    
    Args:
    - directory (str): The directory path to traverse
    """
    def print_directory_recursively(root_dir, depth):
        """
        Recursive function to print the directory structure with headings
        
        Args:
        - root_dir (str): The root directory to start traversal from
        - depth (int): The depth of the current directory in the hierarchy
        """
        for item in sorted(os.listdir(root_dir)):
            item_path = os.path.join(root_dir, item)
            if os.path.isdir(item_path):
                print("#" * (depth) + " " + item)
                print_directory_recursively(item_path, depth + 1)
            elif item.endswith(".en.vtt"):
                print("# " * depth + "- " + item)

    print("# Directory Structure")
    print_directory_recursively(directory, 0)

# Example usage:
directory_path = "/home/al/Documents/courses/Udacity - Artificial Intelligence AI for Trading v1.0.0"
print_directory_structure_with_vtt(directory_path)

In [ ]:
import os
import glob

def print_vtt_files_sorted(directory):
    """
    Traverse the given directory, sort the file names alphabetically,
    and print the file names of files ending with ".en.vtt".
    
    Args:
    - directory (str): The directory path to traverse
    """
    vtt_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".en.vtt"):
                vtt_files.append(os.path.join(root, file))
    
    vtt_files.sort()  # Sort the list of file names alphabetically
    
    for file_path in vtt_files:
        print(file_path)

# Example usage:
directory_path = 
print_vtt_files_sorted(directory_path)
